In [ ]:
import camelot
import tabula
import pandas as pd

# Schritt 1: PDF Verkehrsunfälle

(Für mehr erklärungen siehe Notebook "Verkehrsunfälle in Gemeinden")

Ursprünglich sollte das die definitive Reinschrift sein. Nun beginne ich, in dieser Chose die Datensätze besser zu putzen, damit ich sie gut mergen kann. 

### PDF einlesen

In [ ]:
tables_stream = camelot.read_pdf("Verkehrsunfälle2018.pdf", flavor='stream', pages='all', table_areas=['0,842,595,0'])

### Daten reinigen

In [ ]:
ff = pd.DataFrame() #=meine leere df

headers = ['Gemeinde', 'Sachschaden', 'Personenschaden', 'Getötete', 
           'Schwerverletzte', 'Lebensbedrohlich Verletzte', 
           'Erheblich Verletzte', 'Leichtverletzte'] #=meine Headers

#Und nun durch alles iterieren
#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
for number in range(0,8):
    temporary = tables_stream[number].df #hier sagen wir, immer vorwärts gehen zur nächsten seite
    temporary = temporary[9:] #hier sagen wir: aber im fall erst ab der 9. reihe
    temporary.columns = headers #und dann setz uns bitte noch diese headers, die wir da definitert haben
    frames = [ff, temporary]  #so kleben wir eine tabelle an die andere an. ff ist ein neues frame, temp ist unsere einzelne tabelle oben
    ff = pd.concat(frames)   #mit concat kommts dann in die richtige form

In [ ]:
def clean(elem):
    if elem == "": #wenn das element ein leerer string ist, gibt mir 0 zurück
        return 0
    else:
        return elem #sonst gib mir das element zurück
    
ff = ff.applymap(clean)

In [ ]:
## Die Gemeinden zwischenzeitlich löschen und dann wieder reinnehmen bei der Umwandlung in Zahlen
gem = ff['Gemeinde'] 
del ff['Gemeinde'] #damit schneide ich die Kolonne "Gmeinde" ab

#für das noch überbleibende Dataframe wandle ich alle Strings in Integer
ff = ff.astype(int)

#und setze die Gemeinde-Kolonne wieder rein.
ff['Gemeinde'] = gem

#und definiere auch gleich die Reihenfolge der Headers, was hier aber nur Kosmetik ist
ff = ff[['Gemeinde', 'Sachschaden', 'Personenschaden', 'Getötete', 
           'Schwerverletzte', 'Lebensbedrohlich Verletzte', 
           'Erheblich Verletzte', 'Leichtverletzte']].copy()

In [ ]:
ff = ff.reset_index() #so läuft das von 1 an durch dann, nicht erst ab 9
del ff['index']

In [ ]:
dfunfälle=ff[:-1]

In [ ]:
del dfunfälle['Lebensbedrohlich Verletzte']
del dfunfälle['Erheblich Verletzte']
dfunfälle.tail(5)

In [ ]:
dfunfälle.tail(5)

# Schritt 2: PDF Einwohnerzahlen

### PDF einlesen

In [ ]:
einwohnerzahlen = camelot.read_pdf('EinwohnerzahlenKantonBern.pdf', flavor='stream', pages='all', table_areas=['0,842,595,0'])
einwohnerzahlen

### PDF reinigen

In [ ]:
#Ich lasse die Sache mit den Headers weg, weil die Kolonnenzahl unterschiedlich war
#Ausserdem schneide ich die letzte Zeile auch noch gleich weg.

ffeinwohnerzahlen = pd.DataFrame() #=meine leere df

        
#Und nun durch alles iterieren
#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
for seite in range(0,4):
    temporaer = einwohnerzahlen[seite].df #hier sagen wir, immer vorwärts gehen zur nächsten seite
    temporaer = temporaer[4:99] #hier sagen wir: aber im fall erst ab der 4. reihe
    frameseinwohnerzahlen = [ffeinwohnerzahlen, temporaer]  #so kleben wir eine tabelle an die andere an. ff ist ein neues frame, temp ist unsere einzelne tabelle oben
    ffeinwohnerzahlen = pd.concat(frameseinwohnerzahlen)   #mit concat kommts dann in die richtige formffeinwohnerzahlen = pd.DataFrame() #=meine leere df

In [ ]:
# dann schnappe ich mir nur die zweil Kolonnen raus, die mich interessieren
einwohnerzahlenneu = ffeinwohnerzahlen.iloc[:, [1,5]]

In [ ]:
#jetzt mach ich noch dass die Tabelle bei 0 beginnt und nicht bei 4 (reset index)
einwohnerzahlenneu = einwohnerzahlenneu.reset_index() #so läuft das von 1 an durch dann, nicht erst ab 9
del einwohnerzahlenneu['index'] #und so lösche ich dann die index-kolonne, die index heisst und nun nicht mehr nötig ist

In [ ]:
# und sage denen, wie sie heissen sollen
headerseinwohnerzahlenneu = ['Gemeinde','Einwohnerzahl 2018']
einwohnerzahlenneu.columns = headerseinwohnerzahlenneu

In [ ]:
#Um die beiden Tabellen zu mergen, muss die Gemeindespalte der Einwohner-Tabelle bearbeitet werden.
#In jeder Zelle müssen die ersten vier Stellen abgeschnitten werden.
def cutstring(string):
    string = string[4:]
    return string
var = "301 Aarberg"
cutstring(var)

In [ ]:
#jetzt für alle
einwohnerzahlenneu['Gemeinde'] = einwohnerzahlenneu['Gemeinde'].apply(cutstring)

In [ ]:
einwohnerzahlenneu

# Hier putze ich jetzt diesen Datensatz, weil ich das Zeug dann vielleicht mergen kann, ohne dass diese NaNs kommen

In [ ]:
def geputzt(zeile):
    zeile = zeile.replace('(', '')
    zeile = zeile.replace(')', '')
    zeile = zeile.replace ('BE', '')
    zeile = zeile.replace(' / ', '/').strip() #strip knallt alles raus, das am anfang und am ende ist
    return zeile

In [ ]:
einwohnerzahlenneu['Gemeinde'] = einwohnerzahlenneu['Gemeinde'].apply(geputzt)

In [ ]:
einwohnerzahlenneu['Gemeinde']

In [ ]:
#wenns komplexer wird fuzzywuzzy anwenden

Das ist schon mal ganz gut. Wie ich beim Mergen, dem nächsten Schritt, sehe, hauts mir ein paar Zahlen raus. Das liegt wohl an diesem doofen Apostroph, das verhindert, dass Python die Chose als Integer einordnen kann (jedenfalls hats mir auch ein paarmal eine Fehlermeldung gegeben, als ich das ändern wollte.) Wie ich das löse, finde ich später raus. Zuerst merge ich das hier noch zusammen, damit ich den nächsten Schritt hätte. Die NaNs putze ich dann mir frischerem Kopf wieder raus. Und am besten nehme ich das hier als Grundlage.

# Schritt 3: Mergen

In [ ]:
pd.merge(dfunfälle,einwohnerzahlenneu,
         how ='left',on='Gemeinde')
#geht immer noch nicht, ich muss die klammern auch bei den unfällen rauslöschen

Geht immer noch nicht bei den Klammern. Ich putze also auch noch den anderen Datensatz mit der exakt gleichen Funktion wie oben.

In [ ]:
dfunfälle['Gemeinde'] = dfunfälle['Gemeinde'].apply(geputzt)

In [ ]:
dfDefinitiv = pd.merge(dfunfälle,einwohnerzahlenneu,
         how ='left',on='Gemeinde')

In [ ]:
dfDefinitiv

## Juhui, ich habe eine richtig gemergte Tabelle, voll krass! nun kann ich mit dem Spielen beginnen
Zur Sicherheit mache ich zuerst von oben ein Reinschrift-Notebook, dann kann ich hier mit dem Rumspielen beginnen. 

## Was will ich eigentlich wissen?

Ich will wissen, welche Gemeinde pro Einwohnerzahl am Gefährlichsten ist. Aufpassen muss ich, dass ich nicht einfach alle Zahlen addieren kann punkto Unfälle, sondern wirklich nur die einzelnen Zahlen nehme. 

Arbeiten mit Dreisatz

Wo gibts prozentual gesehen die meisten Toten?
Wo gibts prozentual gesehen am meisten Schwerverletzte?
Wo gibts prozentual gesehen am meisten Tote und Schwerverletzte?
Wo gibts prozentual gesehen am meisten Personenschaden?
Wo gibts prozentual gesehen am meisten Sachschaden?

### Wie soll das aussehen?

Eigentlich möchte ich für jede dieser Fragen eine Tabelle, von denen ich dann die Top10 oder so nehmen kann. Einzeln, damit ich das dann abgeichen kann. 

#### Ein weiergehender Schritt könnte sein
Dass ich auch noch die vergangenen Jahre mitberücksichtige. ACHTUNG: Dort muss ich dann aber immer die damalige Einwohnerzahl reinholen und dann Dataframes machen, die ich gegenspielen kann.

In [ ]:
dfDefinitiv.info()

#### Mist, ich hab jetzt die Einwohnerzahl noch als Objekt drin anstatt als Integer. Ich versuche, das zu ändern

In [ ]:
dfDefinitiv['Einwohnerzahl 2018']

#### Der Grund ist also, dass es so doofe Strichli noch drin hat. Also putze ich doch die auch noch grad raus. 
Ich muss die Spalte zuerst in Strings umwandeln, damit ich das Strichli mit Replace rausnehmen kann. Sonst reklamiert er. 

In [ ]:
dfDefinitiv['Einwohnerzahl 2018'] = dfDefinitiv['Einwohnerzahl 2018'].astype(str)
#Umwandeln in einen String und gleich so definieren

In [ ]:
dfDefinitiv.info()

In [ ]:
def striche_weg(zeile):
    zeile = zeile.replace("'", "").strip()
    return zeile

In [ ]:
dfDefinitiv['Einwohnerzahl 2018'] = dfDefinitiv['Einwohnerzahl 2018'].apply(striche_weg)

In [ ]:
dfDefinitiv['Einwohnerzahl 2018']

### Jetzt hab ich die Striche draussen. Nun schaue ich, ob ich die Einwohnerzahlen in einen Integer wandeln kann

In [ ]:
!pip install fuzzywuzzy[speedup]

In [ ]:
dfDefinitiv['Einwohnerzahl 2018'].astype(int)

Ich hab also irgendwo noch so ein Krüppel-NaN drin. Knurr!!!

In [ ]:
#df = df.dropna()  so könnte ich die nans droppen

In [179]:
dfDefinitiv.sort_values(by='Einwohnerzahl 2018', ascending=False).head(20)

,Gemeinde,Sachschaden,Personenschaden,Getötete,Schwerverletzte,Leichtverletzte,Einwohnerzahl 2018
271,Seehof,0,0,0,0,0,nan
71,Evilard,3,3,0,1,2,nan
261,Schelten,0,0,0,0,0,nan
91,Gsteig,5,2,0,1,1,nan
198,Niederried bei Interlaken,1,1,0,0,2,nan
84,Golaten,0,1,0,0,1,nan
231,Reutigen,0,3,0,0,3,999
15,Bargen,2,4,0,0,4,999
304,Unterlangenegg,3,6,0,0,10,998
258,Saxeten,0,0,0,0,0,99


#### Gwaaaa! Da sind also noch immer sieben nans drin! Ich muss zuerst schauen, weshalb die drin sind. Und wie ich die rausputzen kann. 

In [162]:
einwohnerzahlenneu.sort_values(by='Einwohnerzahl 2018', ascending=False).head(50
                                                                             )

,Gemeinde,Einwohnerzahl 2018
234,Reutigen,999
15,Bargen BE,999
305,Unterlangenegg,998
261,Saxeten,99
60,Diessbach bei Büren,989
91,Gsteig b. Gstaad,978
82,Gampelen,963
330,Wiler bei Utzenstorf,963
296,Trachselwald,960
233,Renan BE,928


Hier empfiehlt Barnaby, mit Fuzzywuzzy weiter zu fahren. Link: http://jonathansoma.com/lede/algorithms-2017/classes/fuzziness-matplotlib/fuzzing-matching-in-pandas-with-fuzzywuzzy/